In [1]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':1,
	'Company_Name':'TREKONIC PRIVATE LIITED ',
	'City':'BANGALORE',
	'Area':['Bull Temple Road','Basavanagudi','Bengaluru Urban'],
	'State':'KARNATAKA',
	'Name':'SHOBHA H',
	'Sur_Name':'H',
	'Trust': False,
	'Pension':False,
 
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'m',
	'GOVT.': False
}

In [2]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':5,
	'Company_Name':'RAIMAXAL PHARAMATEUTICALS PRIVATE LIMITED',
	'City':'SURAT',
	'Area':['Ratanji Nagar','Mithi Khadi','Bhavani Nagar','Limbayat'],
	'State':'Gujarat',
	'Name':'RAJA SHEKAR',
	'Sur_Name':'S',
	'Trust': False,
	'Pension':False,
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'r',
	'GOVT.': False
}

In [19]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':5,
	'Company_Name':'HAXSPLOIT SECURITY SERVICES PRIVATE LIMITED',
	'City':'INDORE',
	'Area':['Kakad','Khandwa'],
	'State':'MADHYA PRADESH',
	'Name':'YASH SHENDGE',
	'Sur_Name':'S',
	'Trust': False,
	'Pension':False,
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'h',
	'GOVT.': False
}

In [36]:
input={
    'C_Type':['Company'],
	# 'S_Type':['Salaried'],
	'TRI':5,
	'Company_Name':'jOURNEY ESCAPE',
	'City':'BANGALORE',
	'Area':['Marenahalli','Saraswathi Nagar'],
	'State':'KARNATAKA',
	'Name':'CHAMPA H N',
	'Sur_Name':'S',
	'Trust': False,
	'Pension':False,
	'Corporate':False,
	'Profession':'ENTREPRENAUR',
	'Exemption':False,
	'C_Name':'j',
	'GOVT.': False
}

In [2]:
default_params = {
    'C_Type': 'Def_C_Type',
    'S_Type': 'Def_S_Type',
    'C_Name': 'All',
    'Sur_Name': 'All',
    'Pension': False,
    'Trust': False,
    'Exemption': False,
    'TRI': 'All',
    'Area': 'All',
    'GOVT.':False,
    'NOT':None
}

In [3]:
import pandas as pd
import ast
lf=pd.read_excel('Mumbai3.xlsx')

In [4]:

lf['C_NAME1'] = lf['C_NAME1'].apply(lambda x: '[' + x + ']')



In [5]:
lf['C_NAME1'] = lf['C_NAME1'].apply(ast.literal_eval)

In [6]:
import ast

# Define a function to convert the string to a dictionary
def convert_to_dict(data_string):
    return ast.literal_eval(data_string)

# Apply the conversion function to the 'ADDITIONAL DESCRIPTION' column
# df1['ADDITIONAL DESCRIPTION'] = df1['ADDITIONAL DESCRIPTION'].apply(convert_to_dict)
# data=convert_to_dict(df1['ADDITIONAL DESCRIPTION'][1])
import traceback

for i, row in lf.iterrows():
    try:
        lf.at[i, 'ADDITIONAL DESCRIPTION'] = convert_to_dict(row['ADDITIONAL DESCRIPTION'])
    except Exception as e:
        print(f"Error occurred at row {i+1}:")
        print(row)
        traceback.print_exc()

In [7]:
filtered_df=lf[lf['CITY']== input['City']]

In [8]:
dropped_count = 0

def filter_data1(input_dict, key, row, default_param=None):
    global dropped_count  # Use the global dropped_count variable
    remaining_data = row['ADDITIONAL DESCRIPTION']['REMAINING_DATA']
    filtered_data = [
        data for data in remaining_data
        if (data.get(key) == input_dict.get(key) or data == input_dict)
        or (default_param is not None and data.get(key) == default_param.get(key))
    ]

    if filtered_data:
        row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = filtered_data
        # print(filtered_data)
        return row
    else:
        dropped_count += 1  # Increment the dropped_count variable
        return None

filtered_df=filtered_df.apply(lambda row: filter_data1(input, 'C_Type', row, default_params), axis=1).dropna()


In [9]:
fd=filtered_df.copy()

In [10]:
fs=fd.apply(lambda row: filter_data1(input, 'Pension', row, default_params), axis=1).dropna()
# fs = pd.concat(fs.tolist())

In [11]:
fs=fs.apply(lambda row: filter_data1(input, 'Govt.', row, default_params), axis=1).dropna()
# fs = pd.concat(fs.tolist())

In [12]:
dropped_count = 0

def filter_data(input_dict, key, row, default_param=None):
    global dropped_count  # Use the global dropped_count variable
    c_namedata = row['C_NAME1']
    remaining_data = row['ADDITIONAL DESCRIPTION']['REMAINING_DATA']
    filtered_data = []

    for data_list in c_namedata:
        for data in data_list:
            if input_dict.get(key).lower().startswith(data):
                filtered_data.append(data_list)

    if filtered_data:
        row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = remaining_data
        return pd.DataFrame(row).transpose()
    
    else:
            dropped_count += 1  # Increment the dropped_count variable
            return None

fss = fs.apply(lambda row: filter_data(input, 'Company_Name', row, default_params), axis=1).dropna()

if fss.empty:
    # default_row = {'C_NAME1': [], 'ADDITIONAL DESCRIPTION': {'REMAINING_DATA': default_params}}
    fss = fs.copy()
else:
    fss = pd.concat(fss.tolist())


In [13]:
dropped_count = 0

def filter_data(input_dict, key, row, default_param=None):
    global dropped_count  # Use the global dropped_count variable
    remaining_data = row['ADDITIONAL DESCRIPTION']['REMAINING_DATA']
    filtered_data = []
    
    for data in remaining_data:
        area_list = data.get(key)
        
        for area in area_list:
            if any(input_area in area for input_area in input_dict[key] for area in area_list):
                filtered_data.append(data)
                break

    if filtered_data:
        row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = filtered_data
        return pd.DataFrame(row).transpose()
    # elif default_param is not None:
    #     row['ADDITIONAL DESCRIPTION']['REMAINING_DATA'] = default_param
    #     return pd.DataFrame(row).transpose()
    else:
        dropped_count += 1
        return None

fz = fss.apply(lambda row: filter_data(input, 'Area', row, default_params), axis=1).dropna()
if fz.empty:
    # default_row = {'C_NAME1': [], 'ADDITIONAL DESCRIPTION': {'REMAINING_DATA': default_params}}
    fz = fss.copy()
else:
    fz = pd.concat(fz.tolist())

In [14]:
fz

,SR. NO.,RCC,AREA CODE,AO TYPE,RANGE CODE,AO NO.,DESCRIPTION,CITY,ADDITIONAL DESCRIPTION,C_NAME,C_NAME1
245,246.0,BLR,KAR,C,131.0,1.0,"CIRCLE 7(1)(1), BANGALORE",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...","[' ""TA to TZ & U""']","[[ta, tb, tc, td, te, tf, tg, th, ti, tj, tk, ..."


In [15]:
fss

,SR. NO.,RCC,AREA CODE,AO TYPE,RANGE CODE,AO NO.,DESCRIPTION,CITY,ADDITIONAL DESCRIPTION,C_NAME,C_NAME1
245,246.0,BLR,KAR,C,131.0,1.0,"CIRCLE 7(1)(1), BANGALORE",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...","[' ""TA to TZ & U""']","[[ta, tb, tc, td, te, tf, tg, th, ti, tj, tk, ..."


In [16]:
fd

,SR. NO.,RCC,AREA CODE,AO TYPE,RANGE CODE,AO NO.,DESCRIPTION,CITY,ADDITIONAL DESCRIPTION,C_NAME,C_NAME1
175,176.0,BLR,KAR,C,113.0,1.0,"CIRCLE 1(1)(1), BANGALORE",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...","[' ""AA to AZ"") ']","[[aa, ab, ac, ad, ae, af, ag, ah, ai, aj, ak, ..."
176,177.0,BLR,KAR,W,113.0,1.0,"WARD 1(1)(1), BANGALORE",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...","[' ""AA to AL""']","[[aa, ab, ac, ad, ae, af, ag, ah, ai, aj, ak, ..."
177,178.0,BLR,KAR,W,224.0,93.0,"WARD 6(3)(1), BANGALORE/",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",['All'],[[All]]
179,180.0,BLR,KAR,W,113.0,92.0,"WARD 1(2)(1), BANGALORE/",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",['All'],[[All]]
183,184.0,BLR,KAR,W,313.0,1.0,"WARD 3(3)(1), BANGALORE",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",['All'],[[All]]
184,185.0,BLR,KAR,C,125.0,1.0,CIRCLE 3(1)(1) BANGALORE,BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...","[' ""F, I""']","[[f, i]]"
185,186.0,BLR,KAR,W,313.0,95.0,"WARD 5(3)(3), BANGALORE/",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",['All'],[[All]]
187,188.0,BLR,KAR,W,125.0,92.0,"WARD 5(1)(1), BANGALORE/",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",['All'],[[All]]
189,190.0,BLR,KAR,W,313.0,93.0,"WARD 5(3)(1), BANGALORE/",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...",['All'],[[All]]
197,198.0,BLR,KAR,C,224.0,1.0,"DC/ACIT CIRCLE 6(1)(1), B",BANGALORE,"{'DEF_AREA': 'NONE', 'REMAINING_DATA': [{'C_Ty...","[' ""SA to SG""']","[[sa, sb, sc, sd, se, sf, sg]]"
